# Structured Output

In [1]:
from devtools import debug
from dotenv import load_dotenv

load_dotenv(verbose=True)

!export PYTHONPATH=":./python"

### Method 1 : provide instruction in the prompt

In [10]:

from langchain.output_parsers import PydanticOutputParser

# IMPORTANT : select Pydantic V1
from langchain_core.pydantic_v1 import BaseModel, Field




In [11]:
"""
The usual "tell me a joke" LLM call.
"""



class Joke(BaseModel):
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")


from python.ai_core.llm import get_llm
from python.ai_core.prompts import def_prompt


class Joke(BaseModel):
    the_joke: str = Field(description="a good joke")
    explanation: str = Field(description="explain why it's funny")
    rate: float = Field(description="rate how the joke is funny between 0 and 5")


parser = PydanticOutputParser(pydantic_object=Joke)

prompt_with_format = """
    tell me  a joke on {topic}     
    --- 
    {format_instructions}"""

structured_prompt = def_prompt(user=prompt_with_format).partial(
    format_instructions=parser.get_format_instructions(),
)

LLM_ID = "gpt_4o_edenai"
parser = PydanticOutputParser(pydantic_object=Joke)
structured_joke = structured_prompt | get_llm(llm_id=LLM_ID) | parser

r = structured_joke.invoke({"topic": "cat"})
debug(r)

2024-07-03 16:01:07.328 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_4o_edenai - configurable: True
2024-07-03 16:01:07.734 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_141811/3804957977.py:38 <module>
    r: Joke(
        the_joke='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!',
        explanation=(
            "The joke is funny because it plays on the double meaning of 'mouse'—a computer mouse and the small rodent"
            ' that cats like to chase.'
        ),
        rate=4.0,
    ) (Joke)


Joke(the_joke='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!', explanation="The joke is funny because it plays on the double meaning of 'mouse'—a computer mouse and the small rodent that cats like to chase.", rate=4.0)

In [3]:
debug(structured_prompt)

/tmp/ipykernel_141811/2639052080.py:1 <module>
    structured_prompt: ChatPromptTemplate(
        input_variables=['topic'],
        partial_variables={
            'format_instructions': (
                'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n'
                '\n'
                'As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strin'
                'gs", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\n'
                'the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"propertie'
                's": {"foo": ["bar", "baz"]}} is not well-formatted.\n'
                '\n'
                'Here is the output schema:\n'
                '```\n'
                '{"properties": {"the_joke": {"title": "The Joke", "description": "a good joke", "type": "string"}, "e'
                'xplanation": {"title": "Explanation", "descr

ChatPromptTemplate(input_variables=['topic'], partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"the_joke": {"title": "The Joke", "description": "a good joke", "type": "string"}, "explanation": {"title": "Explanation", "description": "explain why it\'s funny", "type": "string"}, "rate": {"title": "Rate", "description": "rate how the joke is funny between 0 and 5", "type": "number"}}, "required": ["the_joke", "explanation", "rate"]}\n```'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['format_instr

In [4]:
# You can have a look at the generated prompt:
print(structured_prompt.invoke({"topic": "cat"}).messages[0].content)


tell me  a joke on cat     
--- 
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"the_joke": {"title": "The Joke", "description": "a good joke", "type": "string"}, "explanation": {"title": "Explanation", "description": "explain why it's funny", "type": "string"}, "rate": {"title": "Rate", "description": "rate how the joke is funny between 0 and 5", "type": "number"}}, "required": ["the_joke", "explanation", "rate"]}
```


### Method #2 : Use "with_structured_output"  (bases on function calls)

In [5]:
prompt = "tell me  a joke on {topic}"

chain = def_prompt(prompt) | get_llm().with_structured_output(Joke)
debug(chain.invoke(({"topic": "cat"})))

2024-07-03 14:05:54.989 | INFO     | python.ai_core.llm:get_llm:405 - get LLM : gpt_35_edenai - configurable: True
2024-07-03 14:05:55.369 | INFO     | python.ai_core.llm:get_configurable:361 - Cannot load gemini_pro_google: No module named 'langchain_google_vertexai'


/tmp/ipykernel_141811/3512315679.py:4 <module>
    chain.invoke(({"topic": "cat"})): Joke(
        the_joke='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!',
        explanation=(
            'This joke is funny because it plays on the common association of cats chasing mice. The twist of the cat '
            'sitting on the computer to watch the mouse adds a humorous element.'
        ),
        rate=4.5,
    ) (Joke)


Joke(the_joke='Why was the cat sitting on the computer? Because it wanted to keep an eye on the mouse!', explanation='This joke is funny because it plays on the common association of cats chasing mice. The twist of the cat sitting on the computer to watch the mouse adds a humorous element.', rate=4.5)

##  Assignement (Optional)
Rate the above joke.
Use https://python.langchain.com/v0.1/docs/modules/model_io/output_parsers/types/enum/ 


In [6]:
from enum import Enum


class JokeRater(Enum):
    NOT_SO_GOOD = 0
    GOOD = 1
    VERY_GOOD = 2

